# Chapter 5. Data Class Builders
---

## ToC

[More About @dataclass](#more-about-dataclass)
        
---

## More About @dataclass

he decorator accepts several keyword arguments. This is its signature:

```python
@dataclass(*, init=True, repr=True, eq=True, order=False,
              unsafe_hash=False, frozen=False)
```


![Figure 80](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/80.PNG)